# Analysis for Differential Jet Mass in Z+Jets events with NanoAODv9

In [12]:
import awkward as ak
import numpy as np
import time
import coffea
import uproot
import hist
import vector
print("awkward version ", ak.__version__)
print("coffea version ", coffea.__version__)
from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from collections import defaultdict
import pickle
from distributed.diagnostics.plugin import UploadDirectory
import os
from cms_utils import *

awkward version  1.10.3
coffea version  0.7.22


### Automatically reload any modules used

In [13]:
%load_ext autoreload
%autoreload 2

In [14]:
from smp_utils import *
from cms_utils import *
from response_maker_nanov9_lib import *
from response_maker_nanov9 import *

In [15]:
#f = uproot.open("root://xcache//store/mc/RunIISummer20UL16NanoAODv9/DYJetsToLL_M-50_HT-100to200_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v2/260000/18D0A087-30BD-FE4E-B447-5F493C2D2794.root")

In [16]:
#f["Events"].keys()

In [9]:
from dask.distributed import Client

client = Client("tls://192.168.86.46:8786")
client

client.register_worker_plugin(UploadDirectory("/home/ZJet/QJetMassUproot/", restart=True, update_path=True))
#print(client.run(os.listdir, "dask-worker-space") )
client.scheduler_info()['workers']

/tmp/ipykernel_8443/2090117391.py:6: DeprecationWarning: `Client.register_worker_plugin` has been deprecated; please use `Client.register_plugin` instead
  client.register_worker_plugin(UploadDirectory("/home/ZJet/QJetMassUproot/", restart=True, update_path=True))


CommClosedError: in <TLS (closed) ConnectionPool.register_nanny_plugin local=tls://192.168.86.46:53482 remote=tls://192.168.86.46:8786>: Stream is closed

In [20]:
# client.register_worker_plugin(UploadDirectory("/home/ZJet/QJetMassUproot/", restart=True, update_path=True), nanny=True)
# #print(client.run(os.listdir, "dask-worker-space") )

### If desired, uncomment these to move or remove old pkl files

In [21]:
#! mv qjetmass_zjets_gen.pkl qjetmass_zjets_gen_old.pkl
#! mv qjetmass_zjets_reco.pkl qjetmass_zjets_reco_old.pkl

In [22]:
# client.scheduler_info()['workers']

### If you want to rerun, here is the line. Otherwise keep it commented to just read the pkl file. 

In [11]:
response_maker_nanov9(testing=False, do_gen=True, client=None, prependstr="root://xcache/")
#response_maker_nanov9(testing=False, do_gen=False, client=client)

Running...


/opt/conda/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.0. Please consider upgrading.
  warnings.warn(


ModuleNotFoundError: No module named 'response_maker_nanov9_lib'

In [ ]:
fout = uproot.recreate("skimmed_mc.root")

In [ ]:
response_maker_nanov9(testing=False, do_gen=True, client=None, prependstr="/mnt/data/cms", skimfilename="skimmed_mc.root")

In [ ]:
#response_maker_nanov9(testing=True, do_gen=True, client=client)
response_maker_nanov9(testing=False, do_gen=True, client=None, prependstr="", skimfilename="skimmed_mc")

In [ ]:
f = uproot.open("skimmed_mc.root")

In [ ]:
f = uproot.open("skimmed_data.root")

In [ ]:
#response_maker_nanov9(testing=True, do_gen=False, client=None, prependstr="/mnt/data/cms")

In [ ]:
len(f["Events"]['reco_jet_pt'].arrays())

### Open pkl file for the output histograms

In [ ]:
with open("qjetmass_zjets_gen.pkl", "rb") as f:
    output = pickle.load( f )

In [ ]:
output.keys()

In [ ]:
output["cutflow"]